In [1]:
# !pip install pandas matplotlib numpy

In [2]:
import os
from pathlib import Path
from tabulate import tabulate

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from plotter import *
from plotter.epochstats import EpochStats
from plotter.plotepochstats import PlotRangeQueryStats, PlotEpochStats, plot_total_data_movement
from plotter.plotselectivities import PlotSelectivities, PlotSelectivitiesRangeQuery
from plotter.utils import vanilla_dirname, rqdc_dirname
from plotter.dataclass import TABLE_DATA, Metric

PROJECT_DIR = Path.cwd().parent.parent

prop = font_manager.FontProperties(fname="./plotter/LinLibertine_Mah.ttf")
plt.rcParams['font.family'] = prop.get_name()
plt.rcParams['text.usetex'] = True
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = 24

TAG = "ycsbexperiment"

In [4]:
random_rq_stats = dict()
random_rq_stats_for_rq = dict()

lb = 1 / SIZE_RATIO
EXPDIRNAME = f"{PROJECT_DIR}/.vstats/experiments-{TAG}-U{UPDATES}-E{ENTRY_SIZE}-B{ENTRIES_PER_PAGE}-S{RANGE_QUERIES}-Y{SELECTIVITY}-T{SIZE_RATIO}"   #  -P{NUM_PAGE_PER_FILE}

print(EXPDIRNAME)
rocksdb_dirpath = os.path.join(EXPDIRNAME, "RocksDB")
# rocksdb_tuned_dirpath = os.path.join(EXPDIRNAME, "RocksDBTuned")
# rr_lb0_smlck0_dirpath = os.path.join(EXPDIRNAME, "RangeReduce[lb=0ANDsmlck=0]")
rr_lb0_dirpath = os.path.join(EXPDIRNAME, "RangeReduce[lb=0]")
rr_lb_1_by_T_dirpath = os.path.join(EXPDIRNAME, "RangeReduce[lb=T^-1]")
rr_lb_1_by_T_and_re_1_dirpath = os.path.join(EXPDIRNAME, "RangeReduce[lb=T^-1ANDre=1]")

filesize = ENTRY_SIZE * ENTRIES_PER_PAGE * NUM_PAGE_PER_FILE

rocksdb = EpochStats(rocksdb_dirpath, NUMEPOCHS, filesize)
# rocksdb_tuned = EpochStats(rocksdb_tuned_dirpath, NUMEPOCHS, filesize)
# rr_lb_0_smlck_0 = EpochStats(rr_lb0_smlck0_dirpath, NUMEPOCHS, filesize)
# rr_lb_0 = EpochStats(rr_lb0_dirpath, NUMEPOCHS, filesize)
rr_lb_1_by_T = EpochStats(rr_lb_1_by_T_dirpath, NUMEPOCHS, filesize)
rr_lb_1_by_T_and_re_1 = EpochStats(rr_lb_1_by_T_and_re_1_dirpath, NUMEPOCHS, filesize)

plot_exp = PlotRangeQueryStats(
    {
        "RocksDB": rocksdb.get_rangequerystats(),
        # "RocksDBTuned": rocksdb_tuned.get_rangequerystats(),
        # "RangeReduce[lb=0 & smlck=0]": rr_lb_0_smlck_0.get_rangequerystats(),
        # "RangeReduce[lb=0]": rr_lb_0.get_rangequerystats(),
        "RangeReduce[lb=T^-1]": rr_lb_1_by_T.get_rangequerystats(),
        "RangeReduce[lb=T^-1 & re=1]": rr_lb_1_by_T_and_re_1.get_rangequerystats(),
    }
)
plot_exp.bytes_read_for_each_range_query()
plot_exp.latency_for_each_range_query()
# plot_exp.plot_instructions_executed_per_range_query()
# plot_exp.plot_CPU_cycles_spent_per_range_query()
# plot_exp.cummulative_latency_for_range_queries()

plt.rcParams['font.size'] = 30
metric_exp = PlotEpochStats(
    {
        "RocksDB": rocksdb.get_plotstats(),
        # "RocksDBTuned": rocksdb_tuned.get_plotstats(),
        # "RangeReduce[lb=0 & smlck=0]": rr_lb_0_smlck_0.get_plotstats(),
        # "RangeReduce[lb=0]": rr_lb_0.get_plotstats(),
        "RangeReduce[lb=T^-1]": rr_lb_1_by_T.get_plotstats(),
        "RangeReduce[lb=T^-1 & re=1]": rr_lb_1_by_T_and_re_1.get_plotstats(),
    }
)
metric_exp.plot_total_bytes_written()
metric_exp.plot_database_size()
metric_exp.plot_compaction_debt()
metric_exp.plot_space_amplification()
metric_exp.plot_compaction_read()
metric_exp.plot_workload_exec_time()
plot_total_data_movement(
    {
        "RocksDB": rocksdb.get_plotstats(),
        # "RocksDBTuned": rocksdb_tuned.get_plotstats(),
        # "RangeReduce[lb=0 & smlck=0]": rr_lb_0_smlck_0.get_plotstats(),
        # "RangeReduce[lb=0]": rr_lb_0.get_plotstats(),
        "RangeReduce[lb=T^-1]": rr_lb_1_by_T.get_plotstats(),
        "RangeReduce[lb=T^-1 & re=1]": rr_lb_1_by_T_and_re_1.get_plotstats(),
    },
    {
        "RocksDB": rocksdb.get_rangequerystats(),
        # "RocksDBTuned": rocksdb_tuned.get_rangequerystats(),
        # "RangeReduce[lb=0 & smlck=0]": rr_lb_0_smlck_0.get_rangequerystats(),
        # "RangeReduce[lb=0]": rr_lb_0.get_rangequerystats(),
        "RangeReduce[lb=T^-1]": rr_lb_1_by_T.get_rangequerystats(),
        "RangeReduce[lb=T^-1 & re=1]": rr_lb_1_by_T_and_re_1.get_rangequerystats(),
    },
)

rows = []
ordered_columns = ["Method"] + [metric.value[0] for metric in Metric]

for method, metrics in TABLE_DATA.items():
    row = {"Method": method}
    row.update(metrics)
    rows.append(row)

ordered_rows = []
for row in rows:
    ordered_row = {col: row.get(col, "") for col in ordered_columns}
    ordered_rows.append(ordered_row)

markdown_table = tabulate(ordered_rows, headers="keys", tablefmt="github")
print(markdown_table)

/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/.vstats/experiments-ycsbexperiment-U8388608-E128-B32-S9000-Y0.3-T6


/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/epochstats.py:499: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self._rangequery_stats = self._rangequery_stats.applymap(
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/epochstats.py:499: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self._rangequery_stats = self._rangequery_stats.applymap(
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/epochstats.py:499: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self._rangequery_stats = self._rangequery_stats.applymap(
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/plotepochstats.py:943: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/plotepochstats.py:1065: UserWarning: FigureCanvasAgg

| Method                      | tot. writes (GB)   | s. amp.   | comp. debt. (MB)   | RQ lat. (avg ms)   | overall data mov. (GB)   | w/l exec. time (sec)   | # RR trig   | avg. fsize         | tot. fcount   | avg. bytes read RQ (MB)   | avg. bytes written RR (MB)   | tot. comp. read (Not RR) (GB)   | tail RQ lat. 98   | tail RQ lat. 100   |
|-----------------------------|--------------------|-----------|--------------------|--------------------|--------------------------|------------------------|-------------|--------------------|---------------|---------------------------|------------------------------|---------------------------------|-------------------|--------------------|
| RocksDB                     | 35499341824        | 1.46      | 14046403920        | 1.35               | 1.66                     | 13997017598439         | 0           | 3.720604955706348  | 403           | 185.30377830674914        | nan                          | 34.29                           | 2.8      

/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/plotepochstats.py:469: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/plotepochstats.py:555: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/plotepochstats.py:618: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/shubham/SSD Lab/LSMQueryDrivenCompaction/src/.notebooks/plotter/plotepochstats.py:1352: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
# markdown_table = tabulate(ordered_rows, headers="keys", tablefmt="latex")
# print(markdown_table)